In [24]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [36]:
df = pd.read_excel('train.xlsx')
df

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,26169,20230310-7492-177993190,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,26170,20230625-16563-206126520,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,26171,20220624-7492-137587082,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,26172,20220427-7491-125459150,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


In [3]:
df.dtypes

Unnamed: 0                     int64
№ брони                       object
Номеров                        int64
Стоимость                    float64
Внесена предоплата             int64
Способ оплаты                 object
Дата бронирования     datetime64[ns]
Дата отмены           datetime64[ns]
Заезд                 datetime64[ns]
Ночей                          int64
Выезд                 datetime64[ns]
Источник                      object
Статус брони                  object
Категория номера              object
Гостей                         int64
Гостиница                      int64
dtype: object

In [37]:
df = df.drop(['Unnamed: 0', '№ брони'], axis=1)

In [38]:
df['В какой час бронировали'] = (df['Дата бронирования'].dt.hour + 2) % 24
df['За сколько часов до заезда бронь'] = (df['Заезд'] - df['Дата бронирования']).dt.total_seconds() // 3600 #насколько рано бронь 
df['План по часам'] = (df['Выезд'] - df['Заезд']).dt.total_seconds() // 3600 # сколько в часах по аланм было находиться
df = df.drop(['Дата бронирования', 'Дата отмены','Заезд','Выезд'], axis=1) # уберем ненужные столбцы

In [40]:
for col in ['Источник']:
    df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
    df = df.drop(col, axis=1)
df['Бронь с официального сайта'] = df['Официальный сайт']
df['Бронь со специальных сайтов'] = df['Bronevik.com(new)'] + df['booking.com'] + df['Бронирование из экстранета'] + df['Zabroniryi.ru'] + df['OneTwoTrip'] + df['ostrovok.ru']  + df['booking.com (Booking.com)']+ df['Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")']+ df['Acase.ru (ООО "ПРАНДИУМ")']+ df['Alean.ru (13.10.2023-02.06.2025)']+ df['Alean.ru (03.02.2022-31.01.2023)']+ df['101hotels.com']+ df['Acase.ru (ООО "КАЛЕЙДОСКОП")']+ df['Otello'] + df['expedia.com (A-Hotels.com)']+ df['Alean.ru (31.10.2023-22.06.2025)'] + df['Svoy Hotel']+ df['Alean.ru (20.01.2021-31.01.2022)'] 
df['Бронь по лояльности'] = df['Программа лояльности'] + df['Cuva']
df['Бронь с гигантов'] = df['Яндекс.Путешествия'] + df['Тинькофф Путешествия'] + df['Ozon'] + df['ВКонтакте']

for col in ['Яндекс.Путешествия', 'Официальный сайт', 'Bronevik.com(new)',
       'booking.com', 'Бронирование из экстранета', 'Zabroniryi.ru',
       'OneTwoTrip', 'ostrovok.ru', 'Программа лояльности', 'Cuva',
       'booking.com (Booking.com)', 'Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")',
       'Acase.ru (ООО "ПРАНДИУМ")', 'Alean.ru (13.10.2023-02.06.2025)',
       'Alean.ru (03.02.2022-31.01.2023)', '101hotels.com',
       'Acase.ru (ООО "КАЛЕЙДОСКОП")', 'Alean.ru (16.01.2023-06.09.2024)',
       'Bronevik.com/Bro.Online', 'Тинькофф Путешествия', 'ВКонтакте',
       'Otello', 'expedia.com (A-Hotels.com)', 'Ozon',
       'Alean.ru (31.10.2023-22.06.2025)', 'Svoy Hotel',
       'Alean.ru (20.01.2021-31.01.2022)']:
    df = df.drop(col, axis=1)



df

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Статус брони,Категория номера,Гостей,Гостиница,В какой час бронировали,За сколько часов до заезда бронь,План по часам,Бронь с официального сайта,Бронь со специальных сайтов,Бронь по лояльности,Бронь с гигантов
0,1,25700.0,0,Внешняя система оплаты,3,Отмена,Номер «Стандарт»,2,1,22,186.0,69.0,False,False,False,True
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2,Активный,Номер «Стандарт»,2,1,16,552.0,45.0,True,False,False,False
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2,Активный,Номер «Студия»,2,4,0,472.0,45.0,True,False,False,False
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1,Активный,Номер «Стандарт»,1,3,17,239.0,21.0,False,True,False,False
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2,Активный,Номер «Люкс»,4,1,21,211.0,45.0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2,Активный,Номер «Стандарт»,2,2,19,1485.0,45.0,True,False,False,False
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),3,Активный,Номер «Студия»,3,4,19,117.0,69.0,True,False,False,False
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),4,Активный,Номер «Стандарт»,2,2,21,1123.0,93.0,True,False,False,False
26172,1,6300.0,0,Гарантия банковской картой,1,Отмена,Номер «Стандарт»,2,3,11,1613.0,21.0,False,True,False,False


In [41]:
for col in ['Статус брони']:
    df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
    df = df.drop(col, axis=1)
df['Метрика'] = df['Активный'] + df['Освоб. заранее'] 
for col in ['Активный', 'Неподтвержденные', 'Ожидается оплата',
       'Освоб. заранее', 'Отмена']:
    df = df.drop(col, axis=1)
df

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Категория номера,Гостей,Гостиница,В какой час бронировали,За сколько часов до заезда бронь,План по часам,Бронь с официального сайта,Бронь со специальных сайтов,Бронь по лояльности,Бронь с гигантов,Активный,Неподтвержденные,Ожидается оплата,Освоб. заранее,Отмена
0,1,25700.0,0,Внешняя система оплаты,3,Номер «Стандарт»,2,1,22,186.0,69.0,False,False,False,True,False,False,False,False,True
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2,Номер «Стандарт»,2,1,16,552.0,45.0,True,False,False,False,True,False,False,False,False
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2,Номер «Студия»,2,4,0,472.0,45.0,True,False,False,False,True,False,False,False,False
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1,Номер «Стандарт»,1,3,17,239.0,21.0,False,True,False,False,True,False,False,False,False
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2,Номер «Люкс»,4,1,21,211.0,45.0,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2,Номер «Стандарт»,2,2,19,1485.0,45.0,True,False,False,False,True,False,False,False,False
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),3,Номер «Студия»,3,4,19,117.0,69.0,True,False,False,False,True,False,False,False,False
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),4,Номер «Стандарт»,2,2,21,1123.0,93.0,True,False,False,False,True,False,False,False,False
26172,1,6300.0,0,Гарантия банковской картой,1,Номер «Стандарт»,2,3,11,1613.0,21.0,False,True,False,False,False,False,False,False,True


In [39]:
df['Источник'].unique()

array(['Яндекс.Путешествия', 'Официальный сайт', 'Bronevik.com(new)',
       'booking.com', 'Бронирование из экстранета', 'Zabroniryi.ru',
       'OneTwoTrip', 'ostrovok.ru', 'Программа лояльности', 'Cuva',
       'booking.com (Booking.com)', 'Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")',
       'Acase.ru (ООО "ПРАНДИУМ")', 'Alean.ru (13.10.2023-02.06.2025)',
       'Alean.ru (03.02.2022-31.01.2023)', '101hotels.com',
       'Acase.ru (ООО "КАЛЕЙДОСКОП")', 'Alean.ru (16.01.2023-06.09.2024)',
       'Bronevik.com/Bro.Online', 'Тинькофф Путешествия', 'ВКонтакте',
       'Otello', 'expedia.com (A-Hotels.com)', 'Ozon',
       'Alean.ru (31.10.2023-22.06.2025)', 'Svoy Hotel',
       'Alean.ru (20.01.2021-31.01.2022)'], dtype=object)

In [39]:
df['Источник'].unique()

array(['Яндекс.Путешествия', 'Официальный сайт', 'Bronevik.com(new)',
       'booking.com', 'Бронирование из экстранета', 'Zabroniryi.ru',
       'OneTwoTrip', 'ostrovok.ru', 'Программа лояльности', 'Cuva',
       'booking.com (Booking.com)', 'Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")',
       'Acase.ru (ООО "ПРАНДИУМ")', 'Alean.ru (13.10.2023-02.06.2025)',
       'Alean.ru (03.02.2022-31.01.2023)', '101hotels.com',
       'Acase.ru (ООО "КАЛЕЙДОСКОП")', 'Alean.ru (16.01.2023-06.09.2024)',
       'Bronevik.com/Bro.Online', 'Тинькофф Путешествия', 'ВКонтакте',
       'Otello', 'expedia.com (A-Hotels.com)', 'Ozon',
       'Alean.ru (31.10.2023-22.06.2025)', 'Svoy Hotel',
       'Alean.ru (20.01.2021-31.01.2022)'], dtype=object)

In [39]:
df['Источник'].unique()

array(['Яндекс.Путешествия', 'Официальный сайт', 'Bronevik.com(new)',
       'booking.com', 'Бронирование из экстранета', 'Zabroniryi.ru',
       'OneTwoTrip', 'ostrovok.ru', 'Программа лояльности', 'Cuva',
       'booking.com (Booking.com)', 'Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")',
       'Acase.ru (ООО "ПРАНДИУМ")', 'Alean.ru (13.10.2023-02.06.2025)',
       'Alean.ru (03.02.2022-31.01.2023)', '101hotels.com',
       'Acase.ru (ООО "КАЛЕЙДОСКОП")', 'Alean.ru (16.01.2023-06.09.2024)',
       'Bronevik.com/Bro.Online', 'Тинькофф Путешествия', 'ВКонтакте',
       'Otello', 'expedia.com (A-Hotels.com)', 'Ozon',
       'Alean.ru (31.10.2023-22.06.2025)', 'Svoy Hotel',
       'Alean.ru (20.01.2021-31.01.2022)'], dtype=object)

In [34]:
df = df.drop(['Дата бронирования', 'Дата отмены','Заезд','Выезд','Источник'], axis=1)

KeyError: "['Дата бронирования', 'Дата отмены', 'Заезд', 'Выезд', 'Источник'] not found in axis"

In [33]:
for col in ['Статус брони']:
    df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
    df = df.drop(col, axis=1)

In [16]:
df['Категория номера'].unique()

array(['Номер «Стандарт»', 'Номер «Студия»', 'Номер «Люкс»',
       'Апартаменты с 2 спальнями с отдельным входом',
       'Коттедж с 3 спальнями',
       '1. Апартаменты с 2 спальнями с отдельным входом\n2. Апартаменты с 2 спальнями с отдельным входом',
       '1. Коттедж с 3 спальнями\n2. Коттедж с 3 спальнями',
       'Коттедж с 2 спальнями',
       '1. Номер «Стандарт»\n2. Номер «Стандарт»',
       '1. Номер «Стандарт»\n2. Номер «Стандарт»\n3. Номер «Стандарт»',
       'Номер «Стандарт» для маломобильных групп населения',
       '1. Номер «Люкс»\n2. Номер «Люкс»',
       '1. Номер «Стандарт»\n2. Номер «Люкс»',
       '1. Коттедж с 2 спальнями\n2. Коттедж с 2 спальнями\n3. Коттедж с 2 спальнями',
       '1. Коттедж с 2 спальнями\n2. Коттедж с 2 спальнями',
       '1. Номер «Стандарт»\n2. Номер «Стандарт»\n3. Номер «Стандарт»\n4. Номер «Стандарт» для маломобильных групп населения',
       '1. Коттедж с 3 спальнями\n2. Коттедж с 3 спальнями\n3. Коттедж с 3 спальнями',
       '1. Котте

In [20]:
y = df['Активный']
df = df.drop(['Активный'], axis=1)

In [21]:
df

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Гостей,Гостиница,Час,Разность в часах,Неподтвержденные,...,1. Номер «Стандарт»\n2. Номер «Стандарт»\n3. Номер «Стандарт»\n4. Номер «Стандарт»\n5. Номер «Стандарт»\n6. Номер «Стандарт»,1. Номер «Стандарт»\n2. Номер «Стандарт»\n3. Номер «Стандарт»\n4. Номер «Стандарт» для маломобильных групп населения,1. Номер «Стандарт»\n2. Номер «Стандарт» для маломобильных групп населения,Апартаменты с 2 спальнями с отдельным входом,Коттедж с 2 спальнями,Коттедж с 3 спальнями,Номер «Люкс»,Номер «Стандарт»,Номер «Стандарт» для маломобильных групп населения,Номер «Студия»
0,1,25700.0,0,Внешняя система оплаты,3,2,1,20,186.375000,False,...,False,False,False,False,False,False,False,True,False,False
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2,2,1,14,552.716111,False,...,False,False,False,False,False,False,False,True,False,False
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2,2,4,22,472.008333,False,...,False,False,False,False,False,False,False,False,False,True
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1,1,3,15,239.068611,False,...,False,False,False,False,False,False,False,True,False,False
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2,4,1,19,211.096389,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2,2,2,17,1485.245000,False,...,False,False,False,False,False,False,False,True,False,False
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),3,3,4,17,117.095278,False,...,False,False,False,False,False,False,False,False,False,True
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),4,2,2,19,1123.598611,False,...,False,False,False,False,False,False,False,True,False,False
26172,1,6300.0,0,Гарантия банковской картой,1,2,3,9,1613.069444,False,...,False,False,False,False,False,False,False,True,False,False


In [46]:
df[df["Стоимость"] > 400000]

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Категория номера,Гостей,Гостиница,В какой час бронировали,За сколько часов до заезда бронь,План по часам,Бронь с официального сайта,Бронь со специальных сайтов,Бронь по лояльности,Бронь с гигантов,Активный,Неподтвержденные,Ожидается оплата,Освоб. заранее,Отмена
62,1,640500.0,511900,Банк. карта: Банк Россия (банк. карта),4,Коттедж с 2 спальнями,3,1,13,1851.0,93.0,False,False,True,False,True,False,False,False,False
4356,3,577800.0,98700,Отложенная электронная оплата: Банк Россия (ба...,7,1. Коттедж с 3 спальнями\n2. Коттедж с 3 спаль...,3,1,20,3524.0,165.0,True,False,False,False,False,False,False,False,True
4715,1,571800.0,571800,Банк. карта: Банк Россия (банк. карта),7,Апартаменты с 2 спальнями с отдельным входом,5,1,20,860.0,165.0,True,False,False,False,True,False,False,False,False
4734,1,443800.0,443800,Банк. карта: Банк Россия (банк. карта),3,Коттедж с 2 спальнями,4,1,22,1818.0,69.0,True,False,False,False,False,False,False,False,True
5073,3,460800.0,460800,Банк. карта: Банк Россия (банк. карта),4,1. Апартаменты с 2 спальнями с отдельным входо...,10,1,13,2547.0,93.0,True,False,False,False,True,False,False,False,False
5553,1,422500.0,422500,Банк. карта: Банк Россия (банк. карта),3,Коттедж с 3 спальнями,4,1,20,2541.0,68.0,True,False,False,False,True,False,False,False,False
6161,1,528000.0,528000,Банк. карта: Банк Россия (банк. карта),3,Коттедж с 3 спальнями,7,1,12,2284.0,69.0,False,False,True,False,True,False,False,False,False
6874,1,525000.0,525000,Банк. карта: Банк Россия (банк. карта),7,Апартаменты с 2 спальнями с отдельным входом,4,1,9,1735.0,165.0,True,False,False,False,True,False,False,False,False
6960,2,459126.0,33534,Банк. карта: Банк Россия (банк. карта),13,1. Апартаменты с 2 спальнями с отдельным входо...,6,1,22,810.0,309.0,False,False,True,False,False,False,False,False,True
6980,1,578500.0,578500,Банк. карта: Банк Россия (банк. карта),4,Коттедж с 2 спальнями,5,1,12,1612.0,93.0,True,False,False,False,True,False,False,False,False


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
mean_absolute_percentage_error(lin_reg.predict(x_test), y_test)

In [ ]:
plt.figure(figsize=(40, 30))
plt.bar(data.columns, lin_reg.coef_)

In [ ]:
lin_reg = Lasso()
lin_reg.fit(x_train, y_train)
mean_absolute_percentage_error(lin_reg.predict(x_test), y_test)

In [ ]:
#выведем коэффициенты при признаках
plt.figure(figsize=(40, 30))
plt.bar(data.columns, lin_reg.coef_)